# Import Libraries

In [1]:
import pandas as pd

In [2]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict

# Read Dataset

In [3]:
df = pd.read_json("../data/yelp/yelp.jsonl", lines=True)

In [4]:
df

,ty_id,review_date,score,author,language,title,content,extractions,sentences
0,NW366Ota9w52nYofW99u1g,,-1,FwqVjnpwpcW6fHQxAYigYg,en,,Good cheap place to get away from the heat a t...,"[{'opinion': 'Good cheap', 'aspect': 'place', ...","[[Good, cheap, place, to, get, away, from, the..."
1,NW366Ota9w52nYofW99u1g,,-1,ZNQXrXdCfRX5m08ddcYgtQ,en,,"If you are looking for something to do, take a...","[{'opinion': 'really old', 'aspect': 'theater'...","[[If, you, are, looking, for, something, to, d..."
2,NW366Ota9w52nYofW99u1g,,-1,gm6W0Ys2_b5d9bUcfL8aYw,en,,"Wow, this is a clean and neat establishment! W...","[{'opinion': 'clean', 'aspect': 'establishment...","[[Wow, ,, this, is, a, clean, and, neat, estab..."
3,NW366Ota9w52nYofW99u1g,,-1,iwIJDqCQfxpfnKTYu6vbNQ,en,,This place was hard to find and pretty much a ...,"[{'opinion': 'hard to find', 'aspect': 'place'...","[[This, place, was, hard, to, find, and, prett..."
4,NW366Ota9w52nYofW99u1g,,-1,yq1ZN_38X3hOWrUJ_ZTKCw,en,,"Yep, read the other reviews and decided to try...","[{'opinion': 'OK', 'aspect': 'ticket prices', ...","[[Yep, ,, read, the, other, reviews, and, deci..."
...,...,...,...,...,...,...,...,...,...
1038121,6bgjcFOy4WHMyw62_1V9Pw,,-1,gTJVaTPMyVzjzDhaX447hA,en,,"We ordered two sandwiches, two sodas, and a bo...","[{'opinion': 'mediocre', 'aspect': 'Sandwich',...","[[We, ordered, two, sandwiches, ,, two, sodas,..."
1038122,6bgjcFOy4WHMyw62_1V9Pw,,-1,SzLKZVb24ZlenSBFTBGGTQ,en,,"Mehh... came in at 9pm, one hour before close....","[{'opinion': 'Saddest', 'aspect': 'grilled che...","[[Mehh, ..., came, in, at, 9, pm, ,, one, hour..."
1038123,6bgjcFOy4WHMyw62_1V9Pw,,-1,2Lh31bK4AFis5WKXcutDdQ,en,,I love coming to Panera! Ashley Love and Luke ...,"[{'opinion': 'amazing and friendly', 'aspect':...","[[I, love, coming, to, Panera, !], [Ashley, Lo..."
1038124,6bgjcFOy4WHMyw62_1V9Pw,,-1,EkC8wieEPelR_EzSmQchdQ,en,,So I walk into The Panera Rest. On Ft Apache t...,[],"[[So, I, walk, into, The, Panera, Rest, .], [O..."


## Test

In [5]:
df['extraction_len'] = df['extractions'].apply(lambda x: len(x))
df['sentence_len'] = df['sentences'].apply(lambda x: len(x))

In [6]:
test_idx = 2
print(df['content'].iloc[test_idx])
print(len(df['sentences'].iloc[test_idx]))
df['extractions'].iloc[test_idx]

Wow, this is a clean and neat establishment! We came in for a 3:30 show on Friday and shared the ~600-person theater with 20-something people. The lobby and hall were pretty dark, and the entrance itself was sketchily situated on a backlot-like side road/street/alley, but the bathroom, theater and seating looked and smelled very clean and sterile, and the staff was very courteous. The sound might've been a bit low, but otherwise it was a clean, straightforward and intimate experience, and if you find yourself on the Vegas Strip and looking for a mainstream/Hollywood/blockbuster movie, this is a great find.
4


[{'opinion': 'clean',
  'aspect': 'establishment',
  'sid': 0,
  'asp_start': 8,
  'asp_end': 8,
  'op_start': 5,
  'op_end': 5,
  'attribute': 'restaurant -> atmosphere',
  'sentiment': 'positive'},
 {'opinion': '~600-person',
  'aspect': 'theater',
  'sid': 1,
  'asp_start': 15,
  'asp_end': 15,
  'op_start': 14,
  'op_end': 14,
  'attribute': 'restaurant -> atmosphere',
  'sentiment': 'neutral'},
 {'opinion': 'pretty dark',
  'aspect': 'hall',
  'sid': 2,
  'asp_start': 3,
  'asp_end': 3,
  'op_start': 5,
  'op_end': 6,
  'attribute': 'restaurant -> atmosphere',
  'sentiment': 'negative'},
 {'opinion': 'very courteous',
  'aspect': 'staff',
  'sid': 2,
  'asp_start': 44,
  'asp_end': 44,
  'op_start': 46,
  'op_end': 47,
  'attribute': 'staff',
  'sentiment': 'positive'},
 {'opinion': 'sketchily situated',
  'aspect': 'entrance',
  'sid': 2,
  'asp_start': 10,
  'asp_end': 10,
  'op_start': 13,
  'op_end': 14,
  'attribute': 'restaurant -> atmosphere',
  'sentiment': 'negative'},
 {

# Filter based on reviews

## Select Top Domains

In [7]:
df_yelp_business = pd.read_json("../data/yelp/business.json", lines=True)

In [8]:
df_yelp_business = df_yelp_business.rename(columns={'business_id': 'ty_id'})

In [9]:
df = df.merge(df_yelp_business[['ty_id', 'name', 'categories']], on=['ty_id'])

In [10]:
df['categories'].value_counts().index.tolist()

['Restaurants, Mexican',
 'Mexican, Restaurants',
 'Chinese, Restaurants',
 'Restaurants, Chinese',
 'Food, Coffee & Tea',
 'Restaurants, Pizza',
 'Restaurants, Thai',
 'Coffee & Tea, Food',
 'Beauty & Spas, Nail Salons',
 'Pizza, Restaurants',
 'Nail Salons, Beauty & Spas',
 'Thai, Restaurants',
 'Italian, Restaurants',
 'Restaurants, Italian',
 'Restaurants, Vietnamese',
 'Vietnamese, Restaurants',
 'Food, Ice Cream & Frozen Yogurt',
 'Breakfast & Brunch, Restaurants',
 'Restaurants, Burgers',
 'American (Traditional), Restaurants',
 'Restaurants, American (New)',
 'Sushi Bars, Restaurants, Japanese',
 'Japanese, Restaurants',
 'Steakhouses, Restaurants',
 'Cinema, Arts & Entertainment',
 'Restaurants, Pizza, Italian',
 'Arts & Entertainment, Cinema',
 'Restaurants, Italian, Pizza',
 'Japanese, Sushi Bars, Restaurants',
 'Pizza, Restaurants, Italian',
 'Burgers, Restaurants',
 'Restaurants, American (Traditional)',
 'Indian, Restaurants',
 'Ice Cream & Frozen Yogurt, Food',
 'Italian

In [11]:
len(df[df['categories'].str.contains("Event Planning & Services")])

63675

In [12]:
df['categories_list'] = df['categories'].apply(lambda x: x.split(", ") if pd.notnull(x) else [])

In [13]:
df = df.reset_index()

In [14]:
df

,index,ty_id,review_date,score,author,language,title,content,extractions,sentences,extraction_len,sentence_len,name,categories,categories_list
0,0,NW366Ota9w52nYofW99u1g,,-1,FwqVjnpwpcW6fHQxAYigYg,en,,Good cheap place to get away from the heat a t...,"[{'opinion': 'Good cheap', 'aspect': 'place', ...","[[Good, cheap, place, to, get, away, from, the...",1,4,United Artists Showcase Theatre 8,"Cinema, Arts & Entertainment, Performing Arts","[Cinema, Arts & Entertainment, Performing Arts]"
1,1,NW366Ota9w52nYofW99u1g,,-1,ZNQXrXdCfRX5m08ddcYgtQ,en,,"If you are looking for something to do, take a...","[{'opinion': 'really old', 'aspect': 'theater'...","[[If, you, are, looking, for, something, to, d...",4,6,United Artists Showcase Theatre 8,"Cinema, Arts & Entertainment, Performing Arts","[Cinema, Arts & Entertainment, Performing Arts]"
2,2,NW366Ota9w52nYofW99u1g,,-1,gm6W0Ys2_b5d9bUcfL8aYw,en,,"Wow, this is a clean and neat establishment! W...","[{'opinion': 'clean', 'aspect': 'establishment...","[[Wow, ,, this, is, a, clean, and, neat, estab...",9,4,United Artists Showcase Theatre 8,"Cinema, Arts & Entertainment, Performing Arts","[Cinema, Arts & Entertainment, Performing Arts]"
3,3,NW366Ota9w52nYofW99u1g,,-1,iwIJDqCQfxpfnKTYu6vbNQ,en,,This place was hard to find and pretty much a ...,"[{'opinion': 'hard to find', 'aspect': 'place'...","[[This, place, was, hard, to, find, and, prett...",3,3,United Artists Showcase Theatre 8,"Cinema, Arts & Entertainment, Performing Arts","[Cinema, Arts & Entertainment, Performing Arts]"
4,4,NW366Ota9w52nYofW99u1g,,-1,yq1ZN_38X3hOWrUJ_ZTKCw,en,,"Yep, read the other reviews and decided to try...","[{'opinion': 'OK', 'aspect': 'ticket prices', ...","[[Yep, ,, read, the, other, reviews, and, deci...",6,9,United Artists Showcase Theatre 8,"Cinema, Arts & Entertainment, Performing Arts","[Cinema, Arts & Entertainment, Performing Arts]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038121,1038121,6bgjcFOy4WHMyw62_1V9Pw,,-1,gTJVaTPMyVzjzDhaX447hA,en,,"We ordered two sandwiches, two sodas, and a bo...","[{'opinion': 'mediocre', 'aspect': 'Sandwich',...","[[We, ordered, two, sandwiches, ,, two, sodas,...",5,8,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad..."
1038122,1038122,6bgjcFOy4WHMyw62_1V9Pw,,-1,SzLKZVb24ZlenSBFTBGGTQ,en,,"Mehh... came in at 9pm, one hour before close....","[{'opinion': 'Saddest', 'aspect': 'grilled che...","[[Mehh, ..., came, in, at, 9, pm, ,, one, hour...",2,11,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad..."
1038123,1038123,6bgjcFOy4WHMyw62_1V9Pw,,-1,2Lh31bK4AFis5WKXcutDdQ,en,,I love coming to Panera! Ashley Love and Luke ...,"[{'opinion': 'amazing and friendly', 'aspect':...","[[I, love, coming, to, Panera, !], [Ashley, Lo...",3,5,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad..."
1038124,1038124,6bgjcFOy4WHMyw62_1V9Pw,,-1,EkC8wieEPelR_EzSmQchdQ,en,,So I walk into The Panera Rest. On Ft Apache t...,[],"[[So, I, walk, into, The, Panera, Rest, .], [O...",0,7,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad..."


In [15]:
top_domains = df['categories_list'].explode().value_counts()
top_domains.index.tolist()

['Restaurants',
 'Food',
 'Nightlife',
 'Bars',
 'American (Traditional)',
 'American (New)',
 'Breakfast & Brunch',
 'Sandwiches',
 'Beauty & Spas',
 'Mexican',
 'Pizza',
 'Coffee & Tea',
 'Burgers',
 'Event Planning & Services',
 'Italian',
 'Shopping',
 'Chinese',
 'Japanese',
 'Seafood',
 'Salad',
 'Desserts',
 'Fast Food',
 'Automotive',
 'Sushi Bars',
 'Asian Fusion',
 'Arts & Entertainment',
 'Cafes',
 'Specialty Food',
 'Sports Bars',
 'Steakhouses',
 'Nail Salons',
 'Bakeries',
 'Active Life',
 'Wine & Spirits',
 'Beer',
 'Home Services',
 'Hair Removal',
 'Pubs',
 'Caterers',
 'Local Services',
 'Thai',
 'Ice Cream & Frozen Yogurt',
 'Health & Medical',
 'Hotels & Travel',
 'Chicken Wings',
 'Barbeque',
 'Cocktail Bars',
 'Mediterranean',
 'Vegetarian',
 'Auto Repair',
 'Juice Bars & Smoothies',
 'Wine Bars',
 'Waxing',
 'Hair Salons',
 'Diners',
 'Day Spas',
 'Delis',
 'Skin Care',
 'Lounges',
 'Gluten-Free',
 'Venues & Event Spaces',
 'Vegan',
 'Vietnamese',
 'Indian',
 'Ca

In [16]:
top_domains

Restaurants                 704679
Food                        258992
Nightlife                   180612
Bars                        169001
American (Traditional)      117520
                             ...  
Installment Loans               50
Taxidermy                       50
Opera & Ballet                  50
Workers Compensation Law        50
Title Loans                     50
Name: categories_list, Length: 884, dtype: int64

### Get Top General Domains

In [17]:
new_df = df.explode(['categories_list'])
new_df = new_df.set_index('categories_list').loc[top_domains.index.tolist(),].reset_index()
new_df = new_df.drop_duplicates(subset=['index'])

In [18]:
new_df['categories_list'].value_counts()

Restaurants                     704679
Beauty & Spas                    73428
Food                             69920
Automotive                       38373
Shopping                         34190
Nightlife                        20970
Home Services                    19407
Event Planning & Services        18336
Active Life                      12933
Arts & Entertainment             12094
Health & Medical                 10527
Pets                              9665
Local Services                    6598
Hotels & Travel                   4739
Professional Services              706
Public Services & Government       450
Education                          404
Financial Services                 347
Religious Organizations            137
Local Flavor                       128
Mass Media                          95
Name: categories_list, dtype: int64

### Select top domains

In [19]:
selected_top_domains = ['Restaurants', 'Hotels', 'Beauty & Spas', 'Arts & Entertainment', 'Automotive']
selected_df = df[df['categories_list'].apply(lambda x: any([cat in selected_top_domains for cat in x]))]
selected_df['domain'] = selected_df['categories_list'].apply(lambda x: list(set(x) & set(selected_top_domains)))
selected_df = selected_df[selected_df['domain'].str.len() == 1]
selected_df['domain'] = selected_df['domain'].apply(lambda x: x[0])

/tmp/ipykernel_3119/3855826295.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['domain'] = selected_df['categories_list'].apply(lambda x: list(set(x) & set(selected_top_domains)))


In [20]:
selected_df['domain'].value_counts()

Restaurants             686343
Beauty & Spas            72324
Automotive               41784
Arts & Entertainment     25249
Hotels                    8658
Name: domain, dtype: int64

## Select top businesses per domain 

In [21]:
k = 50
# k = 30

In [22]:
top_k_business = []
for domain in selected_df['domain'].unique():
# for domain in ['Restaurants']:
    top_k_business += selected_df.loc[selected_df['domain'] == domain, 'ty_id'].value_counts().head(k).index.tolist()

In [23]:
selected_df = selected_df[df['ty_id'].isin(top_k_business)]

/tmp/ipykernel_3119/3778154810.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_df = selected_df[df['ty_id'].isin(top_k_business)]


In [24]:
selected_df

,index,ty_id,review_date,score,author,language,title,content,extractions,sentences,extraction_len,sentence_len,name,categories,categories_list,domain
306994,306994,iCCsd62g79PDjswXaqxltw,,-1,S4nbxIsCCYj0VIhRF8BuDg,en,,Hey Mrs. Robinson! \n\nThe Graduate was a fant...,"[{'opinion': 'modern', 'aspect': 'twist', 'sid...","[[Hey, Mrs., Robinson, !, ], [The, Graduate,...",4,6,Graduate Tempe,"Venues & Event Spaces, Event Planning & Servic...","[Venues & Event Spaces, Event Planning & Servi...",Hotels
306995,306995,iCCsd62g79PDjswXaqxltw,,-1,HNmHme33Ar8EnskRVmBypg,en,,"The rooms are decent sized, and the view is aw...","[{'opinion': 'decent sized', 'aspect': 'rooms'...","[[The, rooms, are, decent, sized, ,, and, the,...",7,6,Graduate Tempe,"Venues & Event Spaces, Event Planning & Servic...","[Venues & Event Spaces, Event Planning & Servi...",Hotels
306996,306996,iCCsd62g79PDjswXaqxltw,,-1,AFBb3pYlCVTpBy2ZvvqsiA,en,,My son is a future sun devil and I found this ...,"[{'opinion': 'awesome', 'aspect': 'employees',...","[[My, son, is, a, future, sun, devil, and, I, ...",2,3,Graduate Tempe,"Venues & Event Spaces, Event Planning & Servic...","[Venues & Event Spaces, Event Planning & Servi...",Hotels
306997,306997,iCCsd62g79PDjswXaqxltw,,-1,4tUGHviSJ7RPpxHhbXiuQw,en,,Awesome place to stay when near ASU. Friendly ...,"[{'opinion': 'Awesome', 'aspect': 'place', 'si...","[[Awesome, place, to, stay, when, near, ASU, ....",7,5,Graduate Tempe,"Venues & Event Spaces, Event Planning & Servic...","[Venues & Event Spaces, Event Planning & Servi...",Hotels
306998,306998,iCCsd62g79PDjswXaqxltw,,-1,GLZP_OfjcQLWivre2uF-Zw,en,,Bad hotel. Very run down. Light broken in our ...,"[{'opinion': 'Bad', 'aspect': 'hotel', 'sid': ...","[[Bad, hotel, .], [Very, run, down, .], [Light...",2,11,Graduate Tempe,"Venues & Event Spaces, Event Planning & Servic...","[Venues & Event Spaces, Event Planning & Servi...",Hotels
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038121,1038121,6bgjcFOy4WHMyw62_1V9Pw,,-1,gTJVaTPMyVzjzDhaX447hA,en,,"We ordered two sandwiches, two sodas, and a bo...","[{'opinion': 'mediocre', 'aspect': 'Sandwich',...","[[We, ordered, two, sandwiches, ,, two, sodas,...",5,8,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad...",Restaurants
1038122,1038122,6bgjcFOy4WHMyw62_1V9Pw,,-1,SzLKZVb24ZlenSBFTBGGTQ,en,,"Mehh... came in at 9pm, one hour before close....","[{'opinion': 'Saddest', 'aspect': 'grilled che...","[[Mehh, ..., came, in, at, 9, pm, ,, one, hour...",2,11,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad...",Restaurants
1038123,1038123,6bgjcFOy4WHMyw62_1V9Pw,,-1,2Lh31bK4AFis5WKXcutDdQ,en,,I love coming to Panera! Ashley Love and Luke ...,"[{'opinion': 'amazing and friendly', 'aspect':...","[[I, love, coming, to, Panera, !], [Ashley, Lo...",3,5,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad...",Restaurants
1038124,1038124,6bgjcFOy4WHMyw62_1V9Pw,,-1,EkC8wieEPelR_EzSmQchdQ,en,,So I walk into The Panera Rest. On Ft Apache t...,[],"[[So, I, walk, into, The, Panera, Rest, .], [O...",0,7,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad...",Restaurants


# Extract Opinionated Sentences

In [25]:
len(selected_df['ty_id'].unique())

250

In [26]:
len(selected_df['ty_id'])

35034

In [27]:
selected_df

,index,ty_id,review_date,score,author,language,title,content,extractions,sentences,extraction_len,sentence_len,name,categories,categories_list,domain
306994,306994,iCCsd62g79PDjswXaqxltw,,-1,S4nbxIsCCYj0VIhRF8BuDg,en,,Hey Mrs. Robinson! \n\nThe Graduate was a fant...,"[{'opinion': 'modern', 'aspect': 'twist', 'sid...","[[Hey, Mrs., Robinson, !, ], [The, Graduate,...",4,6,Graduate Tempe,"Venues & Event Spaces, Event Planning & Servic...","[Venues & Event Spaces, Event Planning & Servi...",Hotels
306995,306995,iCCsd62g79PDjswXaqxltw,,-1,HNmHme33Ar8EnskRVmBypg,en,,"The rooms are decent sized, and the view is aw...","[{'opinion': 'decent sized', 'aspect': 'rooms'...","[[The, rooms, are, decent, sized, ,, and, the,...",7,6,Graduate Tempe,"Venues & Event Spaces, Event Planning & Servic...","[Venues & Event Spaces, Event Planning & Servi...",Hotels
306996,306996,iCCsd62g79PDjswXaqxltw,,-1,AFBb3pYlCVTpBy2ZvvqsiA,en,,My son is a future sun devil and I found this ...,"[{'opinion': 'awesome', 'aspect': 'employees',...","[[My, son, is, a, future, sun, devil, and, I, ...",2,3,Graduate Tempe,"Venues & Event Spaces, Event Planning & Servic...","[Venues & Event Spaces, Event Planning & Servi...",Hotels
306997,306997,iCCsd62g79PDjswXaqxltw,,-1,4tUGHviSJ7RPpxHhbXiuQw,en,,Awesome place to stay when near ASU. Friendly ...,"[{'opinion': 'Awesome', 'aspect': 'place', 'si...","[[Awesome, place, to, stay, when, near, ASU, ....",7,5,Graduate Tempe,"Venues & Event Spaces, Event Planning & Servic...","[Venues & Event Spaces, Event Planning & Servi...",Hotels
306998,306998,iCCsd62g79PDjswXaqxltw,,-1,GLZP_OfjcQLWivre2uF-Zw,en,,Bad hotel. Very run down. Light broken in our ...,"[{'opinion': 'Bad', 'aspect': 'hotel', 'sid': ...","[[Bad, hotel, .], [Very, run, down, .], [Light...",2,11,Graduate Tempe,"Venues & Event Spaces, Event Planning & Servic...","[Venues & Event Spaces, Event Planning & Servi...",Hotels
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038121,1038121,6bgjcFOy4WHMyw62_1V9Pw,,-1,gTJVaTPMyVzjzDhaX447hA,en,,"We ordered two sandwiches, two sodas, and a bo...","[{'opinion': 'mediocre', 'aspect': 'Sandwich',...","[[We, ordered, two, sandwiches, ,, two, sodas,...",5,8,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad...",Restaurants
1038122,1038122,6bgjcFOy4WHMyw62_1V9Pw,,-1,SzLKZVb24ZlenSBFTBGGTQ,en,,"Mehh... came in at 9pm, one hour before close....","[{'opinion': 'Saddest', 'aspect': 'grilled che...","[[Mehh, ..., came, in, at, 9, pm, ,, one, hour...",2,11,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad...",Restaurants
1038123,1038123,6bgjcFOy4WHMyw62_1V9Pw,,-1,2Lh31bK4AFis5WKXcutDdQ,en,,I love coming to Panera! Ashley Love and Luke ...,"[{'opinion': 'amazing and friendly', 'aspect':...","[[I, love, coming, to, Panera, !], [Ashley, Lo...",3,5,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad...",Restaurants
1038124,1038124,6bgjcFOy4WHMyw62_1V9Pw,,-1,EkC8wieEPelR_EzSmQchdQ,en,,So I walk into The Panera Rest. On Ft Apache t...,[],"[[So, I, walk, into, The, Panera, Rest, .], [O...",0,7,Panera Bread,"Breakfast & Brunch, Soup, Food, Bagels, Salad,...","[Breakfast & Brunch, Soup, Food, Bagels, Salad...",Restaurants


## Merge Opinions of same sentence into single entry

In [28]:
def extract_opinionated_sentences_from_review(grp):
    extractions = grp['extractions'].iloc[0]
    sentences = grp['sentences'].iloc[0]
    
    opinionated_sid_map = {}
    for opinion in (extractions):
        if opinion['sid'] in opinionated_sid_map:
            opinionated_sid_map[opinion['sid']] += [opinion]
        else:
            opinionated_sid_map[opinion['sid']] = [opinion]
    
    opinionated_sentences = [" ".join(sentences[i]) for i in opinionated_sid_map.keys()]
    opinions = []
    aspects = []
    attributes = []
    sentiments = []
    opinion_aspect_pairs = []
    for sent_opinions in opinionated_sid_map.values():
        opinions += [[opinion['opinion'] for opinion in sent_opinions]]
        aspects += [[opinion['aspect'] for opinion in sent_opinions]]
        attributes += [[opinion['attribute'] for opinion in sent_opinions]]
        sentiments += [[opinion['sentiment'] for opinion in sent_opinions]]
        opinion_aspect_pairs += [[opinion['opinion'] + " " + opinion['aspect'] for opinion in sent_opinions]]
    
    opinion_sent_df = pd.DataFrame({'sentences': opinionated_sentences})
    opinion_sent_df['opinions'] = opinions
    opinion_sent_df['aspects'] = aspects
    opinion_sent_df['opinion_aspect_pairs'] = opinion_aspect_pairs
    opinion_sent_df['sid'] = opinionated_sid_map.keys()
    opinion_sent_df['attributes'] = attributes
    opinion_sent_df['sentiments'] = sentiments
    opinion_sent_df['review_content'] = grp['content'].iloc[0]
    opinion_sent_df.insert(0, 'business_id', grp['ty_id'].iloc[0])
    opinion_sent_df.insert(0, 'name', grp['name'].iloc[0])
    opinion_sent_df.insert(0, 'categories', grp['categories'].iloc[0])
    opinion_sent_df.insert(0, 'categories_list', [grp['categories_list'].iloc[0] for i in range(len(opinion_sent_df))])
    opinion_sent_df.insert(0, 'domain', grp['domain'].iloc[0])
    
    return opinion_sent_df

In [29]:
yelp_sent_df = selected_df.reset_index().groupby(['index']).apply(extract_opinionated_sentences_from_review)

In [30]:
yelp_sent_df

domain                                    categories_list  \
index                                                                       
306994  0       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
        1       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
306995  0       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
        1       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
        2       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
...                ...                                                ...   
1038122 1  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
1038123 0  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
        1  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
1038125 0  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
        1  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   

                                                  categories            name  \
index                                                                          
306994  0  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
        1  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
306995  0  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
        1  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
        2  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
...                                                      ...             ...   
1038122 1  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
1038123 0  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
        1  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
1038125 0  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
        1  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   

                      business_id  \
index                               
306994  0  iCCsd62g79PDjswXaqxltw   
        1  iCCsd62g79PDjswXaqxltw   
306995  0  iCCsd62g79PDjswXaqxltw   
        1  iCCsd62g79PDjswXaqxltw   
        2  iCCsd62g79PDjswXaqxltw   
...                           ...   
1038122 1  6bgjcFOy4WHMyw62_1V9Pw   
1038123 0  6bgjcFOy4WHMyw62_1V9Pw   
        1  6bgjcFOy4WHMyw62_1V9Pw   
1038125 0  6bgjcFOy4WHMyw62_1V9Pw   
        1  6bgjcFOy4WHMyw62_1V9Pw   

                                                   sentences  \
index                                                          
306994  0  Gorgeous 70 's vintage feel with that modern t...   
        1  I especially liked the convenience of the Norm...   
306995  0  The rooms are decent sized , and the view is a...   
        1  Employees for the most part after nice althoug...   
        2  The ' theme ' of the hotel I found to be kind ...   
...                                                      ...   
1038122 1                             Tomato soup was ok .     
1038123 0        The staff here is so amazing and friendly !   
        1                 Great food and wonderful service !   
1038125 0                                Love Panera bread !   
        1                   Everything is always delicious .   

                                     opinions               aspects  \
index                                                                 
306994  0  [modern, boutique style, Gorgeous]  [twist, hotel, feel]   
        1                             [liked]         [convenience]   
306995  0             [decent sized, awesome]         [rooms, view]   
        1                 [metrosexual, nice]      [men, Employees]   
        2                            [cheesy]               [hotel]   
...                                       ...                   ...   
1038122 1                                [ok]         [Tomato soup]   
1038123 0          

In [31]:
yelp_sent_df.to_pickle("../data/yelp/snippext_yelp_sent_df.pkl")

# Sample

**NOTE: In final, select reviews having at least 3 sentences, or opinions to avoid out-of-domain content (i.e nails shop), we target on restaurant only.**

## Sample Top Sentiment Struct Dist (On Restaurant & Hotel)

### Read Data

In [32]:
yelp_sent_df = pd.read_pickle("../data/yelp/snippext_yelp_sent_df.pkl")

In [33]:
yelp_sent_df

domain                                    categories_list  \
index                                                                       
306994  0       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
        1       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
306995  0       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
        1       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
        2       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
...                ...                                                ...   
1038122 1  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
1038123 0  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
        1  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
1038125 0  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
        1  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   

                                                  categories            name  \
index                                                                          
306994  0  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
        1  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
306995  0  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
        1  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
        2  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
...                                                      ...             ...   
1038122 1  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
1038123 0  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
        1  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
1038125 0  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
        1  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   

                      business_id  \
index                               
306994  0  iCCsd62g79PDjswXaqxltw   
        1  iCCsd62g79PDjswXaqxltw   
306995  0  iCCsd62g79PDjswXaqxltw   
        1  iCCsd62g79PDjswXaqxltw   
        2  iCCsd62g79PDjswXaqxltw   
...                           ...   
1038122 1  6bgjcFOy4WHMyw62_1V9Pw   
1038123 0  6bgjcFOy4WHMyw62_1V9Pw   
        1  6bgjcFOy4WHMyw62_1V9Pw   
1038125 0  6bgjcFOy4WHMyw62_1V9Pw   
        1  6bgjcFOy4WHMyw62_1V9Pw   

                                                   sentences  \
index                                                          
306994  0  Gorgeous 70 's vintage feel with that modern t...   
        1  I especially liked the convenience of the Norm...   
306995  0  The rooms are decent sized , and the view is a...   
        1  Employees for the most part after nice althoug...   
        2  The ' theme ' of the hotel I found to be kind ...   
...                                                      ...   
1038122 1                             Tomato soup was ok .     
1038123 0        The staff here is so amazing and friendly !   
        1                 Great food and wonderful service !   
1038125 0                                Love Panera bread !   
        1                   Everything is always delicious .   

                                     opinions               aspects  \
index                                                                 
306994  0  [modern, boutique style, Gorgeous]  [twist, hotel, feel]   
        1                             [liked]         [convenience]   
306995  0             [decent sized, awesome]         [rooms, view]   
        1                 [metrosexual, nice]      [men, Employees]   
        2                            [cheesy]               [hotel]   
...                                       ...                   ...   
1038122 1                                [ok]         [Tomato soup]   
1038123 0          

### Single-aspect only for dataset creation

In [34]:
yelp_sent_df = yelp_sent_df[yelp_sent_df['aspects'].str.len() == 1]

In [35]:
yelp_sent_df['sentiments'].value_counts()

[positive]    44612
[negative]    12193
[neutral]      4021
Name: sentiments, dtype: int64

### Ignore neutral

In [36]:
yelp_sent_df = yelp_sent_df[yelp_sent_df['sentiments'].apply(lambda x: all([sentiment != 'neutral' for sentiment in x]))]

In [37]:
yelp_sent_df

domain                                    categories_list  \
index                                                                       
306994  1       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
306995  3       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
        4       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
306997  0       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
        3       Hotels  [Venues & Event Spaces, Event Planning & Servi...   
...                ...                                                ...   
1038121 4  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
1038122 0  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
1038123 0  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
1038125 0  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
        1  Restaurants  [Breakfast & Brunch, Soup, Food, Bagels, Salad...   

                                                  categories            name  \
index                                                                          
306994  1  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
306995  3  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
        4  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
306997  0  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
        3  Venues & Event Spaces, Event Planning & Servic...  Graduate Tempe   
...                                                      ...             ...   
1038121 4  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
1038122 0  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
1038123 0  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
1038125 0  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   
        1  Breakfast & Brunch, Soup, Food, Bagels, Salad,...    Panera Bread   

                      business_id  \
index                               
306994  1  iCCsd62g79PDjswXaqxltw   
306995  3  iCCsd62g79PDjswXaqxltw   
        4  iCCsd62g79PDjswXaqxltw   
306997  0  iCCsd62g79PDjswXaqxltw   
        3  iCCsd62g79PDjswXaqxltw   
...                           ...   
1038121 4  6bgjcFOy4WHMyw62_1V9Pw   
1038122 0  6bgjcFOy4WHMyw62_1V9Pw   
1038123 0  6bgjcFOy4WHMyw62_1V9Pw   
1038125 0  6bgjcFOy4WHMyw62_1V9Pw   
        1  6bgjcFOy4WHMyw62_1V9Pw   

                                                   sentences  \
index                                                          
306994  1  I especially liked the convenience of the Norm...   
306995  3              Oh and hallway carpet is terrible .     
        4  Overall very pleasant stay and would like to r...   
306997  0              Awesome place to stay when near ASU .   
        3  However , the carpet pattern on the floor leve...   
...                                                      ...   
1038121 4  Table was weird , dirty , and wobbly , like it...   
1038122 0                           - Saddest grilled cheese   
1038123 0        The staff here is so amazing and friendly !   
1038125 0                                Love Panera bread !   
        1                   Everything is always delicious .   

                         opinions           aspects  \
index                                                 
306994  1                 [liked]     [convenience]   
306995  3              [terrible]  [hallway carpet]   
        4         [very pleasant]            [stay]   
306997  0               [Awesome]           [place]   
        3               [hideous]    [floor levels]   
...                           ...               ...   
1038121 4                 [weird]           [Table]   
1038122 0               [Saddest]  [grilled cheese]   
1038123 0  [amazing and friendly]           [staff]   
1038125 0                  [Love]          [Panera]  

In [38]:
yelp_sent_df['sentiments'].value_counts()

[positive]    44612
[negative]    12193
Name: sentiments, dtype: int64

In [39]:
yelp_sent_df.shape

(56805, 13)

### Sample top 100 business per sentiment struct & >= review sent

In [40]:
yelp_sent_df = yelp_sent_df.reset_index()

In [41]:
yelp_sent_df = yelp_sent_df.groupby(['index']).filter(lambda grp: len(grp) > 2)

In [42]:
yelp_sent_df = yelp_sent_df.set_index(['index', 'level_1']).sort_index()

In [43]:
yelp_sent_df

domain  \
index   level_1                
306997  0             Hotels   
        3             Hotels   
        4             Hotels   
307005  0             Hotels   
        1             Hotels   
...                      ...   
1038117 3        Restaurants   
        4        Restaurants   
1038121 2        Restaurants   
        3        Restaurants   
        4        Restaurants   

                                                   categories_list  \
index   level_1                                                      
306997  0        [Venues & Event Spaces, Event Planning & Servi...   
        3        [Venues & Event Spaces, Event Planning & Servi...   
        4        [Venues & Event Spaces, Event Planning & Servi...   
307005  0        [Venues & Event Spaces, Event Planning & Servi...   
        1        [Venues & Event Spaces, Event Planning & Servi...   
...                                                            ...   
1038117 3        [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
        4        [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
1038121 2        [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
        3        [Breakfast & Brunch, Soup, Food, Bagels, Salad...   
        4        [Breakfast & Brunch, Soup, Food, Bagels, Salad...   

                                                        categories  \
index   level_1                                                      
306997  0        Venues & Event Spaces, Event Planning & Servic...   
        3        Venues & Event Spaces, Event Planning & Servic...   
        4        Venues & Event Spaces, Event Planning & Servic...   
307005  0        Venues & Event Spaces, Event Planning & Servic...   
        1        Venues & Event Spaces, Event Planning & Servic...   
...                                                            ...   
1038117 3        Breakfast & Brunch, Soup, Food, Bagels, Salad,...   
        4        Breakfast & Brunch, Soup, Food, Bagels, Salad,...   
1038121 2        Breakfast & Brunch, Soup, Food, Bagels, Salad,...   
        3        Breakfast & Brunch, Soup, Food, Bagels, Salad,...   
        4        Breakfast & Brunch, Soup, Food, Bagels, Salad,...   

                           name             business_id  \
index   level_1                                           
306997  0        Graduate Tempe  iCCsd62g79PDjswXaqxltw   
        3        Graduate Tempe  iCCsd62g79PDjswXaqxltw   
        4        Graduate Tempe  iCCsd62g79PDjswXaqxltw   
307005  0        Graduate Tempe  iCCsd62g79PDjswXaqxltw   
        1        Graduate Tempe  iCCsd62g79PDjswXaqxltw   
...                         ...                     ...   
1038117 3          Panera Bread  6bgjcFOy4WHMyw62_1V9Pw   
        4          Panera Bread  6bgjcFOy4WHMyw62_1V9Pw   
1038121 2          Panera Bread  6bgjcFOy4WHMyw62_1V9Pw   
        3          Panera Bread  6bgjcFOy4WHMyw62_1V9Pw   
        4          Panera Bread  6bgjcFOy4WHMyw62_1V9Pw   

                                                         sentences  \
index   level_1                                                      
306997  0                    Awesome place to stay when near ASU .   
        3        However , the carpet pattern on the floor leve...   
        4               The bar at the bottom has decent servers .   
307005  0                                     Reasonable price .     
        1              The decor is unjust and college themed .      
...                                                            ...   
1038117 3                        Tomato soup is always perfect .     
        4        My favorite location is the Boca Park just bec...   
1038121 2                      Place felt dirty and disorganized .   
        3        Place was deserted on Saturday afternoon , now...   
        4        Table was weird , dirty , and wobbly , like it...   

                         opinions         aspects  \
index   level_1                                    

In [44]:
yelp_sent_df.to_pickle("../data/yelp/snippext_yelp_rest_hotel_sent_df_single_aspect_5_domains_top_50.pkl")

# DRAFT

# Sample

**NOTE: In final, select reviews having at least 3 sentences, or opinions to avoid out-of-domain content (i.e nails shop), we target on restaurant only.**

## Filter Business Name

In [17]:
yelp_sent_df = pd.read_pickle("../data/yelp/snippext_yelp_sent_df.pkl")

In [18]:
yelp_sent_df.reset_index()['index'].value_counts()

988034    15
805167    14
699332    13
209164    13
862635    13
          ..
785551     1
785550     1
346051     1
346055     1
0          1
Name: index, Length: 953366, dtype: int64

In [44]:
df_yelp_business = pd.read_json("../data/yelp/business.json", lines=True)

In [45]:
bid = df_yelp_business['business_id'].tolist()

### Restaurant & Hotel Only

In [46]:
df_yelp_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192604,nqb4kWcOwp8bFxzfvaDpZQ,Sanderson Plumbing,,North Las Vegas,NV,89032,36.213732,-115.177059,5.0,9,1,{'BusinessAcceptsCreditCards': 'True'},"Water Purification Services, Water Heater Inst...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
192605,vY2nLU5K20Pee-FdG0br1g,Chapters,17440 Yonge Street,Newmarket,ON,L3Y 6Y9,44.052658,-79.481850,4.5,3,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Books, Mags, Music & Video, Shopping",None
192606,MiEyUDKTjeci5TMfxVZPpg,Phoenix Pavers,21230 N 22nd St,Phoenix,AZ,85024,33.679992,-112.035569,4.5,14,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Contractors, Landscaping, Mason...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ..."
192607,zNMupayB2jEHVDOji8sxoQ,Beasley's Barber Shop,4406 E Main St,Mesa,AZ,85205,33.416137,-111.735743,4.5,15,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Barbers","{'Tuesday': '8:30-17:30', 'Wednesday': '8:30-1..."


In [47]:
def filter_hotel(example):
    return (pd.notnull(example['categories']) and ("hotel" in example['categories'].lower())) \
    or ("hotel" in example['name'].lower())

test_df = df_yelp_business[df_yelp_business.apply(filter_hotel, axis=1)]
hotel_id = test_df['business_id'].tolist()

In [48]:
def filter_restaurant(example):
    return (pd.notnull(example['categories']) and ("restaurant" in example['categories'].lower())) \
    or ("restaurant" in example['name'].lower())

test_df = df_yelp_business[df_yelp_business.apply(filter_restaurant, axis=1)]
restaurant_id = test_df['business_id'].tolist()

In [49]:
def filter_hotel_restaurant(example):
    return (pd.notnull(example['categories']) and ("restaurant" in example['categories'].lower() or "hotel" in example['categories'].lower())) \
    or ("restaurant" in example['name'].lower() or "hotel" in example['name'].lower())

In [50]:
df_yelp_restaurant_hotel = df_yelp_business[df_yelp_business.apply(filter_hotel_restaurant, axis=1)]
df_yelp_restaurant_hotel

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
11,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W..."
13,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,5981 Andrews Rd,Mentor-on-the-Lake,OH,44060,41.708520,-81.359556,4.0,16,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Italian, Restaurants, Pizza, Chicken Wings","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ..."
17,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119,36.100016,-115.128529,4.0,40,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...","Restaurants, Italian",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192589,ghovD5ZTGDQ5Q2U4ERddWw,Five Guys,3863 Medina Rd,Fairlawn,OH,44333,41.136561,-81.639712,4.0,22,1,"{'RestaurantsPriceRange2': '2', 'HasTV': 'Fals...","Burgers, Restaurants, Fast Food, American (New)","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
192595,h3QErqS3OZgLJ5Tb6-sLyQ,Asia Hut,1450 Kingston Rd,Pickering,ON,L1V 1C1,43.841844,-79.083881,4.5,4,1,"{'BikeParking': 'True', 'RestaurantsAttire': '...","Restaurants, Soup, Chinese, Caribbean","{'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3..."
192596,KnafX7T6qSAmSrLhd709vA,Pho Maisonneuve,948 Boulevard de Maisonneuve Est,Montréal,QC,H2L 1Z1,45.517430,-73.558873,4.0,25,1,"{'GoodForKids': 'True', 'WiFi': 'u'no'', 'Rest...","Vietnamese, Soup, Restaurants","{'Monday': '12:0-21:0', 'Tuesday': '12:0-21:0'..."
192601,jJoPLIhCjkGXy59I4Jl6zw,JW Marriott Phoenix Desert Ridge Resort & Spa,5350 E Marriott Dr,Phoenix,AZ,85054,33.683910,-111.966235,4.0,441,1,"{'BusinessAcceptsBitcoin': 'False', 'GoodForKi...","Golf, Hotels, Day Spas, Hotels & Travel, Event...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [51]:
restaurant_hotel_id = df_yelp_restaurant_hotel['business_id'].tolist()

In [52]:
restaurant_hotel_sent_df = yelp_sent_df[yelp_sent_df['business_id'].isin(restaurant_hotel_id)].reset_index()
restaurant_hotel_sent_df

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,review_content
0,300,0,EnwLYvr22hc0bJOUeSz38A,Great beer and service .,[Great],[beer],[Great beer],0.0,[drink -> alcohol],[positive],Great beer and service. Lots of seating and a ...
1,300,1,EnwLYvr22hc0bJOUeSz38A,Lots of seating and a good atmosphere !,"[good, Lots of]","[atmosphere, seating]","[good atmosphere, Lots of seating]",1.0,"[restaurant -> atmosphere, restaurant -> comfort]","[positive, positive]",Great beer and service. Lots of seating and a ...
2,301,0,EnwLYvr22hc0bJOUeSz38A,"Otherwise though , we did n't get much of a ch...",[out],[beer selection],[out beer selection],1.0,[drink -> alcohol],[neutral],"Tried the Cranberry Wit and the Blueberry, bot..."
3,301,1,EnwLYvr22hc0bJOUeSz38A,The Pot of Pickles was a variety of pickled ve...,[variety of],[pickled veg],[variety of pickled veg],2.0,[food -> vegetarian option],[positive],"Tried the Cranberry Wit and the Blueberry, bot..."
4,302,0,EnwLYvr22hc0bJOUeSz38A,The service we got here was amazing,[amazing],[service],[amazing service],2.0,[staff],[positive],Friends and I went here yesterday night for th...
...,...,...,...,...,...,...,...,...,...,...,...
1886368,1038122,1,6bgjcFOy4WHMyw62_1V9Pw,Tomato soup was ok .,[ok],[Tomato soup],[ok Tomato soup],6.0,[food -> quality],[neutral],"Mehh... came in at 9pm, one hour before close...."
1886369,1038123,0,6bgjcFOy4WHMyw62_1V9Pw,The staff here is so amazing and friendly !,[amazing and friendly],[staff],[amazing and friendly staff],2.0,[staff],[positive],I love coming to Panera! Ashley Love and Luke ...
1886370,1038123,1,6bgjcFOy4WHMyw62_1V9Pw,Great food and wonderful service !,"[Great, wonderful]","[food, service]","[Great food, wonderful service]",4.0,"[food -> quality, staff]","[positive, positive]",I love coming to Panera! Ashley Love and Luke ...
1886371,1038125,0,6bgjcFOy4WHMyw62_1V9Pw,Love Panera bread !,[Love],[Panera],[Love Panera],0.0,[food -> quality],[positive],Love Panera bread! So lucky to have them cater...


Merge to get business name

In [53]:
restaurant_hotel_sent_df = restaurant_hotel_sent_df\
    .merge(df_yelp_restaurant_hotel[['business_id', 'name']], on=['business_id'])

In [54]:
restaurant_hotel_sent_df

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,review_content,name
0,300,0,EnwLYvr22hc0bJOUeSz38A,Great beer and service .,[Great],[beer],[Great beer],0.0,[drink -> alcohol],[positive],Great beer and service. Lots of seating and a ...,Trolley 5
1,300,1,EnwLYvr22hc0bJOUeSz38A,Lots of seating and a good atmosphere !,"[good, Lots of]","[atmosphere, seating]","[good atmosphere, Lots of seating]",1.0,"[restaurant -> atmosphere, restaurant -> comfort]","[positive, positive]",Great beer and service. Lots of seating and a ...,Trolley 5
2,301,0,EnwLYvr22hc0bJOUeSz38A,"Otherwise though , we did n't get much of a ch...",[out],[beer selection],[out beer selection],1.0,[drink -> alcohol],[neutral],"Tried the Cranberry Wit and the Blueberry, bot...",Trolley 5
3,301,1,EnwLYvr22hc0bJOUeSz38A,The Pot of Pickles was a variety of pickled ve...,[variety of],[pickled veg],[variety of pickled veg],2.0,[food -> vegetarian option],[positive],"Tried the Cranberry Wit and the Blueberry, bot...",Trolley 5
4,302,0,EnwLYvr22hc0bJOUeSz38A,The service we got here was amazing,[amazing],[service],[amazing service],2.0,[staff],[positive],Friends and I went here yesterday night for th...,Trolley 5
...,...,...,...,...,...,...,...,...,...,...,...,...
1886368,1038122,1,6bgjcFOy4WHMyw62_1V9Pw,Tomato soup was ok .,[ok],[Tomato soup],[ok Tomato soup],6.0,[food -> quality],[neutral],"Mehh... came in at 9pm, one hour before close....",Panera Bread
1886369,1038123,0,6bgjcFOy4WHMyw62_1V9Pw,The staff here is so amazing and friendly !,[amazing and friendly],[staff],[amazing and friendly staff],2.0,[staff],[positive],I love coming to Panera! Ashley Love and Luke ...,Panera Bread
1886370,1038123,1,6bgjcFOy4WHMyw62_1V9Pw,Great food and wonderful service !,"[Great, wonderful]","[food, service]","[Great food, wonderful service]",4.0,"[food -> quality, staff]","[positive, positive]",I love coming to Panera! Ashley Love and Luke ...,Panera Bread
1886371,1038125,0,6bgjcFOy4WHMyw62_1V9Pw,Love Panera bread !,[Love],[Panera],[Love Panera],0.0,[food -> quality],[positive],Love Panera bread! So lucky to have them cater...,Panera Bread


In [127]:
restaurant_hotel_sent_df.to_pickle("../data/yelp/snippext_yelp_restaurant_hotel_sent_df.pkl")

### Get dataset statistics on number of comments

In [6]:
restaurant_hotel_sent_df = pd.read_pickle("../data/yelp/snippext_yelp_restaurant_hotel_sent_df.pkl")

In [55]:
len(restaurant_hotel_sent_df['index'].unique())

686618

In [56]:
restaurant_hotel_sent_df

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,review_content,name
0,300,0,EnwLYvr22hc0bJOUeSz38A,Great beer and service .,[Great],[beer],[Great beer],0.0,[drink -> alcohol],[positive],Great beer and service. Lots of seating and a ...,Trolley 5
1,300,1,EnwLYvr22hc0bJOUeSz38A,Lots of seating and a good atmosphere !,"[good, Lots of]","[atmosphere, seating]","[good atmosphere, Lots of seating]",1.0,"[restaurant -> atmosphere, restaurant -> comfort]","[positive, positive]",Great beer and service. Lots of seating and a ...,Trolley 5
2,301,0,EnwLYvr22hc0bJOUeSz38A,"Otherwise though , we did n't get much of a ch...",[out],[beer selection],[out beer selection],1.0,[drink -> alcohol],[neutral],"Tried the Cranberry Wit and the Blueberry, bot...",Trolley 5
3,301,1,EnwLYvr22hc0bJOUeSz38A,The Pot of Pickles was a variety of pickled ve...,[variety of],[pickled veg],[variety of pickled veg],2.0,[food -> vegetarian option],[positive],"Tried the Cranberry Wit and the Blueberry, bot...",Trolley 5
4,302,0,EnwLYvr22hc0bJOUeSz38A,The service we got here was amazing,[amazing],[service],[amazing service],2.0,[staff],[positive],Friends and I went here yesterday night for th...,Trolley 5
...,...,...,...,...,...,...,...,...,...,...,...,...
1886368,1038122,1,6bgjcFOy4WHMyw62_1V9Pw,Tomato soup was ok .,[ok],[Tomato soup],[ok Tomato soup],6.0,[food -> quality],[neutral],"Mehh... came in at 9pm, one hour before close....",Panera Bread
1886369,1038123,0,6bgjcFOy4WHMyw62_1V9Pw,The staff here is so amazing and friendly !,[amazing and friendly],[staff],[amazing and friendly staff],2.0,[staff],[positive],I love coming to Panera! Ashley Love and Luke ...,Panera Bread
1886370,1038123,1,6bgjcFOy4WHMyw62_1V9Pw,Great food and wonderful service !,"[Great, wonderful]","[food, service]","[Great food, wonderful service]",4.0,"[food -> quality, staff]","[positive, positive]",I love coming to Panera! Ashley Love and Luke ...,Panera Bread
1886371,1038125,0,6bgjcFOy4WHMyw62_1V9Pw,Love Panera bread !,[Love],[Panera],[Love Panera],0.0,[food -> quality],[positive],Love Panera bread! So lucky to have them cater...,Panera Bread


In [83]:
restaurant_hotel_sent_df.loc[restaurant_hotel_sent_df['business_id'].isin(restaurant_id), 'domain'] = 'restaurant'

In [84]:
restaurant_hotel_sent_df.loc[restaurant_hotel_sent_df['business_id'].isin(hotel_id), 'domain'] = 'hotel'

In [59]:
restaurant_hotel_sent_df['domain'].value_counts()

restaurant    1820821
hotel           65552
Name: domain, dtype: int64

In [16]:
# test_df = restaurant_hotel_sent_df[restaurant_hotel_sent_df['aspects'].str.len() == 1]
# test_df['attributes'] = test_df['attributes'].apply(lambda x: x[0])

In [17]:
# test_df['attributes'].value_counts()

In [18]:
# len(test_df['attributes'].unique())

### Get dataset statistics on number of reviews

Number of business

In [60]:
len(df['ty_id'].unique())

12051

In [66]:
stat_df = df[df['ty_id'].isin(restaurant_hotel_id)].reset_index()
stat_df = stat_df.rename(columns={'ty_id': 'business_id'})
stat_df = stat_df.merge(df_yelp_restaurant_hotel[['business_id', 'name']], on=['business_id'])
stat_df

,index,business_id,review_date,score,author,language,title,content,extractions,sentences,extraction_len,sentence_len,name
0,300,EnwLYvr22hc0bJOUeSz38A,,-1,pfyMZz4vXloX2vEEuvjovA,en,,Great beer and service. Lots of seating and a ...,"[{'opinion': 'Great', 'aspect': 'beer', 'sid':...","[[Great, beer, and, service, .], [Lots, of, se...",3,3,Trolley 5
1,301,EnwLYvr22hc0bJOUeSz38A,,-1,0_g53i0mOcBgN9Ia2abKBA,en,,"Tried the Cranberry Wit and the Blueberry, bot...","[{'opinion': 'out', 'aspect': 'beer selection'...","[[Tried, the, Cranberry, Wit, and, the, Bluebe...",2,5,Trolley 5
2,302,EnwLYvr22hc0bJOUeSz38A,,-1,nWWBnDrUpZqhnXtCuDCUHA,en,,Friends and I went here yesterday night for th...,"[{'opinion': 'amazing', 'aspect': 'service', '...","[[Friends, and, I, went, here, yesterday, nigh...",4,8,Trolley 5
3,303,EnwLYvr22hc0bJOUeSz38A,,-1,n2P300FktYPK6KrNNs7cRA,en,,Kicking ass! These guys have made a great loca...,"[{'opinion': 'great', 'aspect': 'location', 's...","[[Kicking, ass, !], [These, guys, have, made, ...",1,4,Trolley 5
4,304,EnwLYvr22hc0bJOUeSz38A,,-1,4Nto0YEZWeM0IDoxMMFlEA,en,,Great poutine and decent poke. Probably not th...,"[{'opinion': 'Great', 'aspect': 'poutine', 'si...","[[Great, poutine, and, decent, poke, .], [Prob...",5,6,Trolley 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
727066,1038121,6bgjcFOy4WHMyw62_1V9Pw,,-1,gTJVaTPMyVzjzDhaX447hA,en,,"We ordered two sandwiches, two sodas, and a bo...","[{'opinion': 'mediocre', 'aspect': 'Sandwich',...","[[We, ordered, two, sandwiches, ,, two, sodas,...",5,8,Panera Bread
727067,1038122,6bgjcFOy4WHMyw62_1V9Pw,,-1,SzLKZVb24ZlenSBFTBGGTQ,en,,"Mehh... came in at 9pm, one hour before close....","[{'opinion': 'Saddest', 'aspect': 'grilled che...","[[Mehh, ..., came, in, at, 9, pm, ,, one, hour...",2,11,Panera Bread
727068,1038123,6bgjcFOy4WHMyw62_1V9Pw,,-1,2Lh31bK4AFis5WKXcutDdQ,en,,I love coming to Panera! Ashley Love and Luke ...,"[{'opinion': 'amazing and friendly', 'aspect':...","[[I, love, coming, to, Panera, !], [Ashley, Lo...",3,5,Panera Bread
727069,1038124,6bgjcFOy4WHMyw62_1V9Pw,,-1,EkC8wieEPelR_EzSmQchdQ,en,,So I walk into The Panera Rest. On Ft Apache t...,[],"[[So, I, walk, into, The, Panera, Rest, .], [O...",0,7,Panera Bread


In [67]:
stat_df = df[df['ty_id'].isin(hotel_id)].reset_index()
stat_df = stat_df.rename(columns={'ty_id': 'business_id'})
stat_df = stat_df.merge(df_yelp_restaurant_hotel[['business_id', 'name']], on=['business_id'])
stat_df

,index,business_id,review_date,score,author,language,title,content,extractions,sentences,extraction_len,sentence_len,name
0,950,Dcp9djUhDx4ERv8CgZnpDg,,-1,fvtUU-QMeTTclcmGqnFcOw,en,,I stayed here for my last visit in Madison Wis...,"[{'opinion': 'small', 'aspect': 'kitchen', 'si...","[[I, stayed, here, for, my, last, visit, in, M...",3,4,HotelRED
1,951,Dcp9djUhDx4ERv8CgZnpDg,,-1,enOKCt7SqH11i-ewFAJO9Q,en,,Stayed here as part of a visit to University o...,"[{'opinion': 'bare', 'aspect': 'concrete', 'si...","[[Stayed, here, as, part, of, a, visit, to, Un...",9,9,HotelRED
2,952,Dcp9djUhDx4ERv8CgZnpDg,,-1,Sviz0s7_EZuKo8Qgb2AgdA,en,,Do not choose this Hotel if you want to have a...,"[{'opinion': 'not recommend', 'aspect': 'place...","[[Do, not, choose, this, Hotel, if, you, want,...",1,7,HotelRED
3,953,Dcp9djUhDx4ERv8CgZnpDg,,-1,1YeJi0o2IFY2Qp5uUoKsGg,en,,"Clean, modern, friendly people at the front de...","[{'opinion': 'friendly', 'aspect': 'people', '...","[[Clean, ,, modern, ,, friendly, people, at, t...",1,4,HotelRED
4,954,Dcp9djUhDx4ERv8CgZnpDg,,-1,Ib5QonvfWdbOAzpwpQY1qg,en,,In fairness this is a review of the bar. And I...,"[{'opinion': 'love', 'aspect': 'bar', 'sid': 1...","[[In, fairness, this, is, a, review, of, the, ...",3,7,HotelRED
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27621,1037286,EtLM-iRYp3UGOEz4_b0I2Q,,-1,AF4h_fV7xIoeiSzcfRR1jg,en,,Holy crap are these people slow!! Avoid at a...,[],"[[Holy, crap, are, these, people, slow, !, !, ...",0,3,Budget Rent A Car
27622,1037287,EtLM-iRYp3UGOEz4_b0I2Q,,-1,dPO_JosGwOTWwk_HvS3qkw,en,,Checking is ridiculous and the staff could car...,"[{'opinion': 'ridiculous', 'aspect': 'Checking...","[[Checking, is, ridiculous, and, the, staff, c...",1,2,Budget Rent A Car
27623,1037288,EtLM-iRYp3UGOEz4_b0I2Q,,-1,k8BmpFSK69z8_QDKOsefzg,en,,Been in line 20 minutes. I have been told that...,"[{'opinion': 'terrible', 'aspect': 'lines', 's...","[[Been, in, line, 20, minutes, .], [I, have, b...",1,5,Budget Rent A Car
27624,1037289,EtLM-iRYp3UGOEz4_b0I2Q,,-1,mOmH2Gl4XXgLntJ7dGrsjQ,en,,I recently rented a car while in the area for ...,[],"[[I, recently, rented, a, car, while, in, the,...",0,6,Budget Rent A Car


In [69]:
# Reviews with identified ABSA informations
stat_df[stat_df['extractions'].str.len() > 0]

,index,business_id,review_date,score,author,language,title,content,extractions,sentences,extraction_len,sentence_len,name
0,950,Dcp9djUhDx4ERv8CgZnpDg,,-1,fvtUU-QMeTTclcmGqnFcOw,en,,I stayed here for my last visit in Madison Wis...,"[{'opinion': 'small', 'aspect': 'kitchen', 'si...","[[I, stayed, here, for, my, last, visit, in, M...",3,4,HotelRED
1,951,Dcp9djUhDx4ERv8CgZnpDg,,-1,enOKCt7SqH11i-ewFAJO9Q,en,,Stayed here as part of a visit to University o...,"[{'opinion': 'bare', 'aspect': 'concrete', 'si...","[[Stayed, here, as, part, of, a, visit, to, Un...",9,9,HotelRED
2,952,Dcp9djUhDx4ERv8CgZnpDg,,-1,Sviz0s7_EZuKo8Qgb2AgdA,en,,Do not choose this Hotel if you want to have a...,"[{'opinion': 'not recommend', 'aspect': 'place...","[[Do, not, choose, this, Hotel, if, you, want,...",1,7,HotelRED
3,953,Dcp9djUhDx4ERv8CgZnpDg,,-1,1YeJi0o2IFY2Qp5uUoKsGg,en,,"Clean, modern, friendly people at the front de...","[{'opinion': 'friendly', 'aspect': 'people', '...","[[Clean, ,, modern, ,, friendly, people, at, t...",1,4,HotelRED
4,954,Dcp9djUhDx4ERv8CgZnpDg,,-1,Ib5QonvfWdbOAzpwpQY1qg,en,,In fairness this is a review of the bar. And I...,"[{'opinion': 'love', 'aspect': 'bar', 'sid': 1...","[[In, fairness, this, is, a, review, of, the, ...",3,7,HotelRED
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27618,1037283,EtLM-iRYp3UGOEz4_b0I2Q,,-1,wBY3ciAg4hJclD_lZ8RA4g,en,,So I recently rented a car from my local Hertz...,"[{'opinion': 'absolutely', 'aspect': 'service'...","[[So, I, recently, rented, a, car, from, my, l...",4,5,Budget Rent A Car
27619,1037284,EtLM-iRYp3UGOEz4_b0I2Q,,-1,rWm0-VYGj6_Yg2_QTZ0U7A,en,,I used this Budget twice in 2015. Both times w...,"[{'opinion': 'great', 'aspect': 'experience', ...","[[I, used, this, Budget, twice, in, 2015, .], ...",2,2,Budget Rent A Car
27622,1037287,EtLM-iRYp3UGOEz4_b0I2Q,,-1,dPO_JosGwOTWwk_HvS3qkw,en,,Checking is ridiculous and the staff could car...,"[{'opinion': 'ridiculous', 'aspect': 'Checking...","[[Checking, is, ridiculous, and, the, staff, c...",1,2,Budget Rent A Car
27623,1037288,EtLM-iRYp3UGOEz4_b0I2Q,,-1,k8BmpFSK69z8_QDKOsefzg,en,,Been in line 20 minutes. I have been told that...,"[{'opinion': 'terrible', 'aspect': 'lines', 's...","[[Been, in, line, 20, minutes, .], [I, have, b...",1,5,Budget Rent A Car


## Sample Top Sentiment Struct Dist (On Restaurant & Hotel)

### Read Restaurant & Hotel Data

In [71]:
restaurant_hotel_sent_df = pd.read_pickle("../data/yelp/snippext_yelp_restaurant_hotel_sent_df.pkl")

In [87]:
restaurant_hotel_sent_df

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,review_content,name,domain
0,300,0,EnwLYvr22hc0bJOUeSz38A,Great beer and service .,[Great],[beer],[Great beer],0.0,[drink -> alcohol],[positive],Great beer and service. Lots of seating and a ...,Trolley 5,restaurant
1,300,1,EnwLYvr22hc0bJOUeSz38A,Lots of seating and a good atmosphere !,"[good, Lots of]","[atmosphere, seating]","[good atmosphere, Lots of seating]",1.0,"[restaurant -> atmosphere, restaurant -> comfort]","[positive, positive]",Great beer and service. Lots of seating and a ...,Trolley 5,restaurant
2,301,0,EnwLYvr22hc0bJOUeSz38A,"Otherwise though , we did n't get much of a ch...",[out],[beer selection],[out beer selection],1.0,[drink -> alcohol],[neutral],"Tried the Cranberry Wit and the Blueberry, bot...",Trolley 5,restaurant
3,301,1,EnwLYvr22hc0bJOUeSz38A,The Pot of Pickles was a variety of pickled ve...,[variety of],[pickled veg],[variety of pickled veg],2.0,[food -> vegetarian option],[positive],"Tried the Cranberry Wit and the Blueberry, bot...",Trolley 5,restaurant
4,302,0,EnwLYvr22hc0bJOUeSz38A,The service we got here was amazing,[amazing],[service],[amazing service],2.0,[staff],[positive],Friends and I went here yesterday night for th...,Trolley 5,restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886368,1038122,1,6bgjcFOy4WHMyw62_1V9Pw,Tomato soup was ok .,[ok],[Tomato soup],[ok Tomato soup],6.0,[food -> quality],[neutral],"Mehh... came in at 9pm, one hour before close....",Panera Bread,restaurant
1886369,1038123,0,6bgjcFOy4WHMyw62_1V9Pw,The staff here is so amazing and friendly !,[amazing and friendly],[staff],[amazing and friendly staff],2.0,[staff],[positive],I love coming to Panera! Ashley Love and Luke ...,Panera Bread,restaurant
1886370,1038123,1,6bgjcFOy4WHMyw62_1V9Pw,Great food and wonderful service !,"[Great, wonderful]","[food, service]","[Great food, wonderful service]",4.0,"[food -> quality, staff]","[positive, positive]",I love coming to Panera! Ashley Love and Luke ...,Panera Bread,restaurant
1886371,1038125,0,6bgjcFOy4WHMyw62_1V9Pw,Love Panera bread !,[Love],[Panera],[Love Panera],0.0,[food -> quality],[positive],Love Panera bread! So lucky to have them cater...,Panera Bread,restaurant


In [73]:
restaurant_hotel_sent_df[restaurant_hotel_sent_df['name'].str.contains("Taco Bell")]

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,review_content,name
46066,30399,0,cJ5i4V6_VKjkOeEHD4ENeQ,The food is amazing here and they never mess m...,[amazing],[food],[amazing food],0.0,[food -> quality],[positive],The food is amazing here and they never mess m...,Taco Bell
46067,30399,1,cJ5i4V6_VKjkOeEHD4ENeQ,After about 6 pm and later the drive thru line...,[so long],[drive thru line],[so long drive thru line],2.0,[wait-time],[negative],The food is amazing here and they never mess m...,Taco Bell
46068,30399,2,cJ5i4V6_VKjkOeEHD4ENeQ,The staff is very friendly although they stand...,[very friendly],[staff],[very friendly staff],6.0,[staff],[positive],The food is amazing here and they never mess m...,Taco Bell
46069,30400,0,cJ5i4V6_VKjkOeEHD4ENeQ,Great customer service ( especially from the g...,[Great],[customer],[Great customer],1.0,[staff],[positive],yummy !!\n\nGreat customer service (especially...,Taco Bell
46070,30400,1,cJ5i4V6_VKjkOeEHD4ENeQ,"fast , good , hot food ! !",[hot],[food],[hot food],3.0,[food -> quality],[positive],yummy !!\n\nGreat customer service (especially...,Taco Bell
...,...,...,...,...,...,...,...,...,...,...,...,...
46151,30444,2,cJ5i4V6_VKjkOeEHD4ENeQ,"food was typical , I think the highlight was t...","[typical, highlight]","[food, frozen drink]","[typical food, highlight frozen drink]",6.0,"[food -> quality, drink -> quality]","[neutral, positive]",waited 30 whole minutes in drive through line ...,Taco Bell
46152,30445,0,cJ5i4V6_VKjkOeEHD4ENeQ,This Taco Bell has the longest wait time ever !,[longest],[wait time],[longest wait time],0.0,[wait-time],[negative],This Taco Bell has the longest wait time ever!...,Taco Bell
46153,30446,0,cJ5i4V6_VKjkOeEHD4ENeQ,My soft tacos were soggy and when I opened my ...,[soggy],[soft tacos],[soggy soft tacos],3.0,[food -> quality],[negative],Okay TACO BELL IS MY GO TO WHEN I WORK LATE SH...,Taco Bell
46154,30447,0,cJ5i4V6_VKjkOeEHD4ENeQ,"Not as cheap as it was back in the day , but s...",[good],[food],[good food],2.0,[food -> quality],[positive],It is a Taco Bell. The restaurant that was sup...,Taco Bell


In [89]:
yelp_sent_df = restaurant_hotel_sent_df

### Single-aspect only for dataset creation

In [90]:
yelp_sent_df = yelp_sent_df[yelp_sent_df['aspects'].str.len() == 1]

In [91]:
yelp_sent_df['sentiments'].value_counts()

[positive]    1067414
[negative]     305792
[neutral]      107414
Name: sentiments, dtype: int64

### Top

In [92]:
# yelp_sent_df[[ 'domain', 'business_id']].value_counts()

In [93]:
# k = 100
# top_k_business = yelp_sent_df['business_id'].value_counts().head(k).index.tolist()
# top_k_business

In [94]:
# k = 100
# top_k_business = yelp_sent_df['business_id'].value_counts().head(k).index.tolist()
# yelp_sent_df = yelp_sent_df[yelp_sent_df['business_id'].isin(top_k_business)]

In [95]:
k = 100
top_k_business = yelp_sent_df.loc[yelp_sent_df['domain'] == 'restaurant', 'business_id'].value_counts().head(int(k/2)).index.tolist()
top_k_business += yelp_sent_df.loc[yelp_sent_df['domain'] == 'hotel', 'business_id'].value_counts().head(int(k/2)).index.tolist()
yelp_sent_df = yelp_sent_df[yelp_sent_df['business_id'].isin(top_k_business)]

In [96]:
yelp_sent_df

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,review_content,name,domain
752208,459745,1,WOuE6HtLVTRothIN-zy4Pw,Prices fluctuate greatly and depending on the ...,[fluctuate],[Prices],[fluctuate Prices],2.0,[value-for-money],[negative],We usually stay here while we are in Pittsburg...,Kimpton Hotel Monaco Pittsburgh,hotel
752209,459745,2,WOuE6HtLVTRothIN-zy4Pw,Downtown Pittsburgh is starting to really revi...,[great],[location],[great location],3.0,[location],[positive],We usually stay here while we are in Pittsburg...,Kimpton Hotel Monaco Pittsburgh,hotel
752210,459745,3,WOuE6HtLVTRothIN-zy4Pw,Only a few miles from other great neighborhood...,[great],[neighborhoods],[great neighborhoods],4.0,[restaurant -> atmosphere],[positive],We usually stay here while we are in Pittsburg...,Kimpton Hotel Monaco Pittsburgh,hotel
752212,459746,1,WOuE6HtLVTRothIN-zy4Pw,The rooms are adorably decorated and they were...,[adorably decorated],[rooms],[adorably decorated rooms],2.0,[restaurant -> atmosphere],[positive],Absolutely fantastic in all ways. Trevor the b...,Kimpton Hotel Monaco Pittsburgh,hotel
752213,459747,0,WOuE6HtLVTRothIN-zy4Pw,You barely have to leave the hotel with a bier...,[great],[restaurant],[great restaurant],3.0,[food -> quality],[positive],This hotel is what decor dreams are made of! ...,Kimpton Hotel Monaco Pittsburgh,hotel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885525,1037790,0,GETVAz5SCk8rFziexV54Aw,Eager staff that merits your attention .,[Eager],[staff],[Eager staff],0.0,[staff],[positive],Eager staff that merits your attention. Unfor...,No Què No Mexican Grill,restaurant
1885526,1037790,1,GETVAz5SCk8rFziexV54Aw,Unfortunately the promising menu comes across ...,[promising],[menu],[promising menu],1.0,[food -> variety],[negative],Eager staff that merits your attention. Unfor...,No Què No Mexican Grill,restaurant
1885527,1037790,2,GETVAz5SCk8rFziexV54Aw,This point is accentuated when you go across t...,[masterfully executed],[food],[masterfully executed food],2.0,[food -> quality],[positive],Eager staff that merits your attention. Unfor...,No Què No Mexican Grill,restaurant
1885528,1037791,0,GETVAz5SCk8rFziexV54Aw,I 've been looking for a good Mexican Food Res...,[good],[Mexican Food Restaurant],[good Mexican Food Restaurant],0.0,[food -> quality],[positive],I've been looking for a good Mexican Food Rest...,No Què No Mexican Grill,restaurant


Ignore neutral

In [97]:
yelp_sent_df = yelp_sent_df[yelp_sent_df['sentiments'].apply(lambda x: all([sentiment != 'neutral' for sentiment in x]))]

In [98]:
yelp_sent_df

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,review_content,name,domain
752208,459745,1,WOuE6HtLVTRothIN-zy4Pw,Prices fluctuate greatly and depending on the ...,[fluctuate],[Prices],[fluctuate Prices],2.0,[value-for-money],[negative],We usually stay here while we are in Pittsburg...,Kimpton Hotel Monaco Pittsburgh,hotel
752209,459745,2,WOuE6HtLVTRothIN-zy4Pw,Downtown Pittsburgh is starting to really revi...,[great],[location],[great location],3.0,[location],[positive],We usually stay here while we are in Pittsburg...,Kimpton Hotel Monaco Pittsburgh,hotel
752210,459745,3,WOuE6HtLVTRothIN-zy4Pw,Only a few miles from other great neighborhood...,[great],[neighborhoods],[great neighborhoods],4.0,[restaurant -> atmosphere],[positive],We usually stay here while we are in Pittsburg...,Kimpton Hotel Monaco Pittsburgh,hotel
752212,459746,1,WOuE6HtLVTRothIN-zy4Pw,The rooms are adorably decorated and they were...,[adorably decorated],[rooms],[adorably decorated rooms],2.0,[restaurant -> atmosphere],[positive],Absolutely fantastic in all ways. Trevor the b...,Kimpton Hotel Monaco Pittsburgh,hotel
752213,459747,0,WOuE6HtLVTRothIN-zy4Pw,You barely have to leave the hotel with a bier...,[great],[restaurant],[great restaurant],3.0,[food -> quality],[positive],This hotel is what decor dreams are made of! ...,Kimpton Hotel Monaco Pittsburgh,hotel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885525,1037790,0,GETVAz5SCk8rFziexV54Aw,Eager staff that merits your attention .,[Eager],[staff],[Eager staff],0.0,[staff],[positive],Eager staff that merits your attention. Unfor...,No Què No Mexican Grill,restaurant
1885526,1037790,1,GETVAz5SCk8rFziexV54Aw,Unfortunately the promising menu comes across ...,[promising],[menu],[promising menu],1.0,[food -> variety],[negative],Eager staff that merits your attention. Unfor...,No Què No Mexican Grill,restaurant
1885527,1037790,2,GETVAz5SCk8rFziexV54Aw,This point is accentuated when you go across t...,[masterfully executed],[food],[masterfully executed food],2.0,[food -> quality],[positive],Eager staff that merits your attention. Unfor...,No Què No Mexican Grill,restaurant
1885528,1037791,0,GETVAz5SCk8rFziexV54Aw,I 've been looking for a good Mexican Food Res...,[good],[Mexican Food Restaurant],[good Mexican Food Restaurant],0.0,[food -> quality],[positive],I've been looking for a good Mexican Food Rest...,No Què No Mexican Grill,restaurant


In [99]:
yelp_sent_df['sentiments'].value_counts()

[positive]    25568
[negative]     6516
Name: sentiments, dtype: int64

In [100]:
top_sentiment_pattern = yelp_sent_df['sentiments'].value_counts().head(10)
top_sentiment_pattern

[positive]    25568
[negative]     6516
Name: sentiments, dtype: int64

In [101]:
yelp_sent_df.shape

(32084, 13)

### Sample top 100 business per sentiment struct & >= review sent

In [102]:
top_sentiment_pattern = top_sentiment_pattern.index.tolist()
top_sentiment_pattern

[['positive'], ['negative']]

In [103]:
yelp_sent_df[yelp_sent_df['sentiments'].isin(top_sentiment_pattern)]

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,review_content,name,domain
752208,459745,1,WOuE6HtLVTRothIN-zy4Pw,Prices fluctuate greatly and depending on the ...,[fluctuate],[Prices],[fluctuate Prices],2.0,[value-for-money],[negative],We usually stay here while we are in Pittsburg...,Kimpton Hotel Monaco Pittsburgh,hotel
752209,459745,2,WOuE6HtLVTRothIN-zy4Pw,Downtown Pittsburgh is starting to really revi...,[great],[location],[great location],3.0,[location],[positive],We usually stay here while we are in Pittsburg...,Kimpton Hotel Monaco Pittsburgh,hotel
752210,459745,3,WOuE6HtLVTRothIN-zy4Pw,Only a few miles from other great neighborhood...,[great],[neighborhoods],[great neighborhoods],4.0,[restaurant -> atmosphere],[positive],We usually stay here while we are in Pittsburg...,Kimpton Hotel Monaco Pittsburgh,hotel
752212,459746,1,WOuE6HtLVTRothIN-zy4Pw,The rooms are adorably decorated and they were...,[adorably decorated],[rooms],[adorably decorated rooms],2.0,[restaurant -> atmosphere],[positive],Absolutely fantastic in all ways. Trevor the b...,Kimpton Hotel Monaco Pittsburgh,hotel
752213,459747,0,WOuE6HtLVTRothIN-zy4Pw,You barely have to leave the hotel with a bier...,[great],[restaurant],[great restaurant],3.0,[food -> quality],[positive],This hotel is what decor dreams are made of! ...,Kimpton Hotel Monaco Pittsburgh,hotel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885525,1037790,0,GETVAz5SCk8rFziexV54Aw,Eager staff that merits your attention .,[Eager],[staff],[Eager staff],0.0,[staff],[positive],Eager staff that merits your attention. Unfor...,No Què No Mexican Grill,restaurant
1885526,1037790,1,GETVAz5SCk8rFziexV54Aw,Unfortunately the promising menu comes across ...,[promising],[menu],[promising menu],1.0,[food -> variety],[negative],Eager staff that merits your attention. Unfor...,No Què No Mexican Grill,restaurant
1885527,1037790,2,GETVAz5SCk8rFziexV54Aw,This point is accentuated when you go across t...,[masterfully executed],[food],[masterfully executed food],2.0,[food -> quality],[positive],Eager staff that merits your attention. Unfor...,No Què No Mexican Grill,restaurant
1885528,1037791,0,GETVAz5SCk8rFziexV54Aw,I 've been looking for a good Mexican Food Res...,[good],[Mexican Food Restaurant],[good Mexican Food Restaurant],0.0,[food -> quality],[positive],I've been looking for a good Mexican Food Rest...,No Què No Mexican Grill,restaurant


In [104]:
dfs = []
for pattern in top_sentiment_pattern:
    df_sentiment_pattern = yelp_sent_df[yelp_sent_df['sentiments'].isin([pattern])]
#     df_sentiment_pattern = df_sentiment_pattern[df_sentiment_pattern['business_id'].isin(top_k_business)]
    df_sentiment_pattern = df_sentiment_pattern.groupby(['index']).filter(lambda grp: len(grp) > 2)
#     df_sentiment_pattern = df_sentiment_pattern.sample(3000, random_state=42)
    dfs += [df_sentiment_pattern]

In [105]:
sample_sent_df = pd.concat(dfs)

In [106]:
sample_sent_df

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,review_content,name,domain
752221,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],[love hotel],0.0,[restaurant -> atmosphere],[positive],I am in love with this hotel.\n\nThe staff is ...,Kimpton Hotel Monaco Pittsburgh,hotel
752222,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],[amazing staff],1.0,[staff],[positive],I am in love with this hotel.\n\nThe staff is ...,Kimpton Hotel Monaco Pittsburgh,hotel
752223,459751,2,WOuE6HtLVTRothIN-zy4Pw,Everyone is so warm and friendly and eager to ...,[warm],[Everyone],[warm Everyone],2.0,[drink -> quality],[positive],I am in love with this hotel.\n\nThe staff is ...,Kimpton Hotel Monaco Pittsburgh,hotel
752224,459751,3,WOuE6HtLVTRothIN-zy4Pw,Also - their food is amazing !,[amazing],[food],[amazing food],6.0,[food -> quality],[positive],I am in love with this hotel.\n\nThe staff is ...,Kimpton Hotel Monaco Pittsburgh,hotel
752225,459752,0,WOuE6HtLVTRothIN-zy4Pw,"Nice , clean rooms .",[clean],[rooms],[clean rooms],0.0,[restaurant -> atmosphere],[positive],"Nice, clean rooms. Pricing is fair. Decor is v...",Kimpton Hotel Monaco Pittsburgh,hotel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885301,1037703,3,GETVAz5SCk8rFziexV54Aw,My boyfriend ordered the chicken burrito and w...,[not as disappointed],[chicken burrito],[not as disappointed chicken burrito],6.0,[food -> quality],[negative],I got this order delivered so it has nothing t...,No Què No Mexican Grill,restaurant
1885378,1037734,1,GETVAz5SCk8rFziexV54Aw,The food was horrible .,[horrible],[food],[horrible food],1.0,[food -> quality],[negative],I went here for my first time this week on my ...,No Què No Mexican Grill,restaurant
1885379,1037734,2,GETVAz5SCk8rFziexV54Aw,We ordered $ 100 worth of food between 4 of us...,[no],[food],[no food],2.0,[food -> quality],[negative],I went here for my first time this week on my ...,No Què No Mexican Grill,restaurant
1885380,1037734,3,GETVAz5SCk8rFziexV54Aw,"The carne asada platter was old , dry and tough .",[old],[carne asada platter],[old carne asada platter],3.0,[food -> healthiness],[negative],I went here for my first time this week on my ...,No Què No Mexican Grill,restaurant


In [107]:
sample_sent_df = sample_sent_df.set_index(['index', 'level_1']).sort_index()

In [108]:
sample_sent_df

business_id  \
index   level_1                           
459751  0        WOuE6HtLVTRothIN-zy4Pw   
        1        WOuE6HtLVTRothIN-zy4Pw   
        2        WOuE6HtLVTRothIN-zy4Pw   
        3        WOuE6HtLVTRothIN-zy4Pw   
459752  0        WOuE6HtLVTRothIN-zy4Pw   
...                                 ...   
1037787 3        GETVAz5SCk8rFziexV54Aw   
        4        GETVAz5SCk8rFziexV54Aw   
1037789 0        GETVAz5SCk8rFziexV54Aw   
        1        GETVAz5SCk8rFziexV54Aw   
        3        GETVAz5SCk8rFziexV54Aw   

                                                         sentences  \
index   level_1                                                      
459751  0                         I am in love with this hotel .     
        1                              The staff is just amazing !   
        2        Everyone is so warm and friendly and eager to ...   
        3                           Also - their food is amazing !   
459752  0                                     Nice , clean rooms .   
...                                                            ...   
1037787 3        Everyone in my family was very happy with thei...   
        4        Overheard over customers remarking on how grea...   
1037789 0        I am always looking for authentic Mexican food...   
        1           Great service , friendly , clean and tasty .     
        3                                Full bar and tvs inside .   

                     opinions         aspects      opinion_aspect_pairs  sid  \
index   level_1                                                                
459751  0              [love]         [hotel]              [love hotel]  0.0   
        1           [amazing]         [staff]           [amazing staff]  1.0   
        2              [warm]      [Everyone]           [warm Everyone]  2.0   
        3           [amazing]          [food]            [amazing food]  6.0   
459752  0             [clean]         [rooms]             [clean rooms]  0.0   
...                       ...             ...                       ...  ...   
1037787 3        [very happy]          [food]         [very happy food]  6.0   
        4             [great]          [food]              [great food]  7.0   
1037789 0         [authentic]  [Mexican food]  [authentic Mexican food]  0.0   
        1             [Great]       [service]           [Great service]  2.0   
        3              [Full]           [bar]                [Full bar]  6.0   

                                 attributes  sentiments  \
index   level_1                                           
459751  0        [restaurant -> atmosphere]  [positive]   
        1                           [staff]  [positive]   
        2                [drink -> quality]  [positive]   
        3                 [food -> quality]  [positive]   
459752  0        [restaurant -> atmosphere]  [positive]   
...                                     ...         ...   
1037787 3                 [food -> quality]  [positive]   
        4                 [food -> quality]  [positive]   
1037789 0                 [food -> quality]  [positive]   
        1                           [staff]  [positive]   
        3           [restaurant -> comfort]  [positive]   

                                                    review_content  \
index   level_1                                                      
459751  0        I am in love with this hotel.\n\nThe staff is ...   
        1        I am in love with this hotel.\n\nThe staff is ...   
        2        I am in love with this hotel.\n\nThe staff is ...   
        3        I am in love with this hotel.\n\nThe staff is ...   
459752  0        Nice, clean rooms. Pricing is fair. Decor is v...   
...                                                            ...   
1037787 3        This place was wonderful! Perfect service. I h...   
        4        This place was wonderful! Perfect service. I h...   
1037789 0        I am always looking for authent

In [109]:
sample_sent_df[sample_sent_df['domain'] == 'hotel']

business_id  \
index   level_1                           
459751  0        WOuE6HtLVTRothIN-zy4Pw   
        1        WOuE6HtLVTRothIN-zy4Pw   
        2        WOuE6HtLVTRothIN-zy4Pw   
        3        WOuE6HtLVTRothIN-zy4Pw   
459752  0        WOuE6HtLVTRothIN-zy4Pw   
...                                 ...   
1026684 2        6Pk9PJOD7EWd1tv3lk0ajA   
        4        6Pk9PJOD7EWd1tv3lk0ajA   
1026685 0        6Pk9PJOD7EWd1tv3lk0ajA   
        1        6Pk9PJOD7EWd1tv3lk0ajA   
        4        6Pk9PJOD7EWd1tv3lk0ajA   

                                                         sentences  \
index   level_1                                                      
459751  0                         I am in love with this hotel .     
        1                              The staff is just amazing !   
        2        Everyone is so warm and friendly and eager to ...   
        3                           Also - their food is amazing !   
459752  0                                     Nice , clean rooms .   
...                                                            ...   
1026684 2                The staff was very friendly and helpful .   
        4        I would stay there again because of the excell...   
1026685 0                                      This place is FAB .   
        1        I loved the landscaping   and decor around the...   
        4        Check the website for great deals not availabl...   

                                    opinions        aspects  \
index   level_1                                               
459751  0                             [love]        [hotel]   
        1                          [amazing]        [staff]   
        2                             [warm]     [Everyone]   
        3                          [amazing]         [food]   
459752  0                            [clean]        [rooms]   
...                                      ...            ...   
1026684 2        [very friendly and helpful]        [staff]   
        4                        [excellent]      [service]   
1026685 0                              [FAB]        [place]   
        1                            [loved]  [landscaping]   
        4                            [great]        [deals]   

                              opinion_aspect_pairs  sid  \
index   level_1                                           
459751  0                             [love hotel]  0.0   
        1                          [amazing staff]  1.0   
        2                          [warm Everyone]  2.0   
        3                           [amazing food]  6.0   
459752  0                            [clean rooms]  0.0   
...                                            ...  ...   
1026684 2        [very friendly and helpful staff]  2.0   
        4                      [excellent service]  6.0   
1026685 0                              [FAB place]  0.0   
        1                      [loved landscaping]  1.0   
        4                            [great deals]  6.0   

                                 attributes  sentiments  \
index   level_1                                           
459751  0        [restaurant -> atmosphere]  [positive]   
        1                           [staff]  [positive]   
        2                [drink -> quality]  [positive]   
        3                 [food -> quality]  [positive]   
459752  0        [restaurant -> atmosphere]  [positive]   
...                                     ...         ...   
1026684 2                           [staff]  [positive]   
        4                           [staff]  [positive]   
1026685 0                  [recommendation]  [positive]   
        1        [restaurant -> atmosphere]  [positive]   
        4                           [staff]  [positive]   

                                                    review_content  \
index   level_1                                                      
459751  0        I am in love with this hotel.\n\nThe staff i

In [110]:
sample_sent_df['domain'].value_counts()

restaurant    9849
hotel         6751
Name: domain, dtype: int64

In [111]:
sample_sent_df['domain'].value_counts()

restaurant    9849
hotel         6751
Name: domain, dtype: int64

In [121]:
sample_sent_df.to_pickle("../data/yelp/snippext_yelp_rest_hotel_sent_df_single_aspect_top_50_50.pkl")

In [112]:
sample_sent_df['attributes'].value_counts()

[food -> quality]              5725
[restaurant -> atmosphere]     3974
[staff]                        2547
[recommendation]               1189
[drink -> quality]              616
[wait-time]                     467
[drink -> alcohol]              407
[value-for-money]               390
[food-quantity]                 378
[food -> variety]               338
[restaurant -> comfort]         226
[location]                      153
[food -> healthiness]           130
[food -> vegetarian option]      56
[good-for-groups]                 4
Name: attributes, dtype: int64